In [1]:
import numpy as np, random, operator
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
class Ciudad:
    def __init__(self, x, y):
        self.x=x
        self.y=y
    def distancia(self, Ciudad):
        xDist = abs(self.x - Ciudad.x)
        yDist = abs(self.y - Ciudad.y)
        distancia = np.sqrt ( (xDist ** 2) + (yDist ** 2))
        return distancia
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) +")"

In [3]:
class Fitness:
    def __init__(self, ruta):
        self.ruta = ruta
        self.distancia = 0
        self.fitness = 0.0
        
    def distanciaRuta(self):
        if self.distancia == 0:
            pathDistancia = 0
            for i in range(0, len(self.ruta)):
                fromCiudad = self.ruta[i]
                toCiudad = None
                if i + 1 < len(self.ruta):
                    toCiudad = self.ruta[i + 1]
                else:
                    toCiudad = self.ruta[0]
                pathDistancia += fromCiudad.distancia(toCiudad)
            self.distancia = pathDistancia
        return self.distancia
        
    def rutaFitness(self):
        if self.fitness ==0:
            self.fitness = 1 / float(self.distanciaRuta())
        return self.fitness
                

In [4]:
def crearRuta(listaCiudades):
    ruta = random.sample(listaCiudades, len(listaCiudades))
    return ruta

In [5]:
def poblacionInicial(popSize, listaCiudades):
    poblacion = []
    
    for i in range(0, popSize):
        poblacion.append(crearRuta(listaCiudades))
    return poblacion

In [6]:
def rankRutas(poblacion):
    resultadosFitness = {}
    for i in range(0, len(poblacion)):
        resultadosFitness[i] = Fitness(poblacion[i]).rutaFitness()
    return sorted(resultadosFitness.items(), key = operator.itemgetter(1),reverse = True)

In [7]:
def seleccion(popRanked, eliteSize):
    resultadosSeleccion = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        resultadosSeleccion.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100 * random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                resultadosSeleccion.append(popRanked[i][0])
                break
    return resultadosSeleccion
def matingPool(pobacion, resultadosSeleccion):
    matingpool = []
    for i in range(0, len(resultadosSeleccion)):
        index = resultadosSeleccion[i]
        matingpool.append(pobacion[index])
    return matingpool

In [8]:
def cruzamiento(padre1, padre2):
    hijo = []
    hijoP1 = []
    hijoP2 = []
    
    genA = int(random.random() * len(padre1))
    genB = int(random.random() * len(padre1))
    
    genInicio = min(genA, genB)
    genFin = max(genA, genB)
    
    for i in range(genInicio, genFin):
        hijoP1.append(padre1[i])
    
    hijoP2 = [item for item in padre2 if item not in hijoP1]
    
    hijo = hijoP1 + hijoP2
    return hijo

In [9]:
def poblacionCruzada(matingpool, eliteSize):
    hijos = []
    tamano = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))
    for i in range(0,eliteSize):
        hijos.append(matingpool[i])
        
    for i in range(0, tamano):
        hijo = cruzamiento(pool[i], pool[len(matingpool)-i-1])
        hijos.append(hijo)
    return hijos

In [10]:
def mutacion(individuo, tasaMutacion):
    for swapped in range(len(individuo)):
        if (random.random() < tasaMutacion):
            swapWith = int(random.random() * len(individuo))
        
            ciudad1 = individuo[swapped]
            ciudad2 = individuo[swapWith]
        
            individuo[swapped] = ciudad2
            individuo[swapWith] = ciudad1
    return individuo

In [11]:
def mutacionPoblacion(poblacion, tasaMutacion):
    popMutada = []
    
    for ind in range(0, len(poblacion)):
        indMutada = mutacion(poblacion[ind], tasaMutacion)
        popMutada.append(indMutada)
    return popMutada

In [12]:
def sigGeneracion(actGeneracion, eliteSize, tasaMutacion):
    popRanked = rankRutas(actGeneracion)
    resultadosSeleccion = seleccion(popRanked, eliteSize)
    matingpool = matingPool(actGeneracion, resultadosSeleccion)
    hijos = poblacionCruzada(matingpool, eliteSize)
    sigGeneracion = mutacionPoblacion(hijos, tasaMutacion)
    return sigGeneracion

In [13]:
def algoritmogenetico(poblacion, popSize, eliteSize, tasaMutacion, generaciones):
    pop = poblacionInicial(popSize, poblacion)
    print("Distancia Inicial:" + str(1 / rankRutas(pop)[0][1]))
    
    for i in range(0, generaciones):
        pop = sigGeneracion(pop, eliteSize, tasaMutacion)
        
    print("Distancia Final:" + str(1 / rankRutas(pop)[0][1]))
    indiceMejorRuta = rankRutas(pop)[0][0]
    mejorRuta = pop[indiceMejorRuta]
    return mejorRuta

In [14]:
listaCiudades = []

for i in range(0,25):
    listaCiudades.append(Ciudad(x=int(random.random()* 200), y=int(random.random() * 200)))



In [15]:
algoritmogenetico(poblacion=listaCiudades, popSize=100, eliteSize=20, tasaMutacion=0.01, generaciones=500)

Distancia Inicial:2024.2664112606917
Distancia Final:920.8898060126834


[(3,88),
 (9,84),
 (16,59),
 (60,66),
 (48,42),
 (78,20),
 (92,34),
 (112,36),
 (140,61),
 (191,56),
 (157,36),
 (102,65),
 (94,59),
 (95,81),
 (85,132),
 (121,144),
 (137,122),
 (146,147),
 (171,147),
 (191,187),
 (139,166),
 (112,197),
 (37,194),
 (34,130),
 (39,102)]

In [154]:
#algoritmogenetico(poblacion=listaciudades, popSize=100, eliteSize=20, tasaMutacion=0.01, generaciones=500)